# 📊 Progetto: Analisi del Sentiment sul Bitcoin
Autrice: Mariagiulia Scialdone

## 🔹 Sezione 1 – Caricamento e preparazione dei dati

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import re

# Dataset tweet
tweet_url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/bitcoin_tweets.csv"
df_tweets = pd.read_csv(tweet_url)

# Dataset prezzi BTC
price_url = "https://github.com/Profession-AI/progetti-big-data/raw/main/Analisi%20del%20consenso%20sul%20Bitcoin/BTC-USD.csv"
df_price = pd.read_csv(price_url)


## 🔹 Sezione 2 – Pulizia dei dati

In [ ]:

df_tweets["created_at"] = pd.to_datetime(df_tweets["created_at"])
df_tweets.drop_duplicates(subset="tweet_id", inplace=True)

def clean_text(text):
    text = re.sub(r"http\S+|www\S+|https\S+", "", str(text))
    text = re.sub(r"[^A-Za-z\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text

df_tweets["clean_text"] = df_tweets["content"].apply(clean_text)


## 🔹 Sezione 3 – Analisi del Sentiment

In [ ]:

df_tweets["polarity"] = df_tweets["clean_text"].apply(lambda text: TextBlob(text).sentiment.polarity)
df_tweets["sentiment"] = df_tweets["polarity"].apply(lambda x: "positive" if x > 0 else ("negative" if x < 0 else "neutral"))


## 🔹 Sezione 4 – Analisi temporale del sentiment

In [ ]:

df_tweets.set_index("created_at", inplace=True)
daily_sentiment = df_tweets.resample("D").agg({
    "polarity": "mean",
    "tweet_id": "count",
    "sentiment": lambda x: x.value_counts().to_dict()
})
daily_sentiment.rename(columns={"tweet_id": "tweet_count"}, inplace=True)
daily_sentiment["positive"] = daily_sentiment["sentiment"].apply(lambda x: x.get("positive", 0))
daily_sentiment["negative"] = daily_sentiment["sentiment"].apply(lambda x: x.get("negative", 0))
daily_sentiment["neutral"] = daily_sentiment["sentiment"].apply(lambda x: x.get("neutral", 0))


## 🔹 Sezione 5 – Correlazione con il prezzo BTC

In [ ]:

df_price["Date"] = pd.to_datetime(df_price["Date"])
daily_sentiment = daily_sentiment.reset_index()
merged_df = pd.merge(daily_sentiment, df_price, left_on="created_at", right_on="Date", how="inner")

correlation = merged_df["polarity"].corr(merged_df["Close"])
print(f"Correlazione tra polarità e prezzo BTC: {correlation:.4f}")


## 🔹 Sezione 6 – Engagement tweet positivi/negativi

In [ ]:

engagement_df = df_tweets.groupby("sentiment")[["retweet_count", "like_count"]].mean().reset_index()
engagement_df


## 🔹 Sezione 7 – Conclusioni

Durante l’analisi ho osservato come il sentiment sui tweet legati al Bitcoin può dare un’indicazione sull’umore generale degli utenti.

Anche se la correlazione con il prezzo non è fortissima, ci sono indicazioni utili per l’analisi del mercato.

I tweet negativi hanno mostrato un engagement spesso maggiore, soprattutto in termini di retweet, suggerendo che la negatività attira più attenzione.
